In [2]:
'''
https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
'''

'\nhttps://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html\nhttps://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456\nhttp://ruder.io/deep-learning-nlp-best-practices/\nhttps://skymind.ai/wiki/word2vec\nhttps://medium.com/@ppasumarthi_69210/word-embeddings-in-keras-be6bb3092831\nhttps://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526\nhttps://medium.com/@dcameronsteinke/tf-idf-vs-word-embedding-a-comparison-and-code-tutorial-5ba341379ab0\nhttps://medium.com/@b.terryjack/nlp-everything-about-word-embeddings-9ea21f51ccfe\nhttps://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795\n'

In [3]:
from __future__ import print_function

import csv
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('rslp')

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/konpyro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/konpyro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/konpyro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package rslp to /home/konpyro/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [4]:
# X: input, Y:output
X = []
Y = []



empty = 0
with open('MoodyLyricsFull4Q.csv') as file:
    reader = csv.reader(file, delimiter = ',')
    count = 0
    for row in reader:
        if count > 0:
            if row[4] == '':
                empty = empty + 1
                continue
            X.append(row[4])
            Y.append(row[3])
        count = count + 1
print(empty)

53


In [5]:
#Y = np.concatenate(y_train, y_test)
y_t = np.zeros((len(Y), 2))
for i  in range(len(Y)):
    if Y[i] == 'happy':
        y_t[i][:] = [1, 1]
    elif Y[i] == 'angry':
        y_t[i][:] = [1, -1]
    elif Y[i] == 'sad':
        y_t[i][:] = [-1, -1]
    elif Y[i] == 'relaxed':
        y_t[i][:] = [ -1, 1]
Y = y_t

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [7]:
t = Tokenizer()
total_lyrics = x_train + x_test
t.fit_on_texts(total_lyrics)

max_length = max([len(s.split()) for s in total_lyrics])
vocab_size = len(t.word_index) + 1
EMBEDDING_DIM = 100

x_train_tokens = t.texts_to_sequences(x_train)
x_test_tokens = t.texts_to_sequences(x_test)

x_train_pad = pad_sequences(x_train_tokens, padding='post')
x_test_pad = pad_sequences(x_test_tokens, padding='post')

In [8]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

lyric_lines = list()
lines = X

for line in lines:
    tokens = word_tokenize(line)

    tokens = [w.lower() for w in tokens]

    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    words = [word for word in stripped if word.isalpha()]

    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    lyric_lines.append(words)

In [9]:
t = Tokenizer()
t.fit_on_texts(lyric_lines)
sequences = t.texts_to_sequences(lyric_lines)

word_index = t.word_index
print('Found %s unique tokens.' %len(word_index))

review_pad = pad_sequences(sequences, maxlen=max_length)
sentiment = np.asarray(Y)
print('Shape of lyric tensor:', review_pad.shape)
print('Shape of sentiment tensor;', sentiment.shape)

Found 20634 unique tokens.
Shape of lyric tensor: (1947, 1015)
Shape of sentiment tensor; (1947, 2)


In [10]:
import os

embeddings_index = {}
f = open(os.path.join('', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [11]:
word_index = t.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        

In [12]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

Batch size, epochs cross validation

In [13]:
def create_model():
    model = Sequential()
    model.add(embedding_layer)
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='tanh'))
    
    model.compile(loss='categorical_hinge', optimizer='adam', metrics=['accuracy'])
    
    return model
        
seed = 7

X = review_pad
Y = sentiment

model = KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [16, 32, 64, 128]
epochs = [5, 10, 15, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=3, cv=3)
grid_result = grid.fit(X, Y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/5
1298/1298 [==============================] - 75s 58ms/step - loss: 0.9298 - accuracy: 0.5886
Epoch 2/5
1298/1298 [==============================] - 73s 56ms/step - loss: 0.8160 - accuracy: 0.5770
Epoch 3/5
1298/1298 [==============================] - 81s 63ms/step - loss: 0.7668 - accuracy: 0.5778
Epoch 4/5
1298/1298 [==============================] - 73s 56ms/step - loss: 0.7398 - accuracy: 0.5570
Epoch 5/5
649/649 [==============================] - 10s 16ms/step
Epoch 1/5
1298/1298 [==============================] - 73s 57ms/step - loss: 0.9192 - accuracy: 0.5416
Epoch 2/5
1298/1298 [==============================] - 75s 58ms/step - loss: 0.8450 - accuracy: 0.5092
Epoch 3/5
1298/1298 [==============================] - 75s 57ms/step - loss: 0.7963 - accuracy: 0.5185
Epoch 4/5
1298/1298 [==============================] - 71s 55ms/step - loss: 0.7485 - accuracy: 0.5624
Epoch 5/5
649/649 [==============================] - 10s 16ms/step
Epoch 1/5
1298/1298 [=====================

In [18]:
print(grid_result.best_params_['batch_size'])

16


Optimaizer cross validation

In [ ]:
def create_model():
    model = Sequential()
    model.add(embedding_layer)
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='tanh'))
    
    model.compile(loss='categorical_hinge', optimizer=optimizer, metrics=['accuracy'])
    
    return model
        
seed = 7

X = review_pad
Y = sentiment

batch_size = grid_result.best_params_['batch_size']
epochs = grid_result.best_params_['epochs']
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)

optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=3, cv=3)
grid_result = grid.fit(X, Y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Number of neurons cross validation

In [ ]:
optimizer = grid_result.best_params_['optimizer']

def create_model():
    model = Sequential()
    model.add(embedding_layer)
    model.add(LSTM(neurons, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='tanh'))
    
    model.compile(loss='categorical_hinge', optimizer=optimizer, metrics=['accuracy'])
    
    return model
        
seed = 7

X = review_pad
Y = sentiment

batch_size = grid_result.best_params_['batch_size']
epochs = grid_result.best_params_['epochs']
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)

neurons = [8, 16, 32, 64, 128]
param_grid = dict(neurons=neurons)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=3, cv=3)
grid_result = grid.fit(X, Y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))